<a href="https://colab.research.google.com/github/Alphambarushimana/Master-thesis-/blob/main/Kopi_af_LSTM_code_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Netflix stock pridiction 

In [50]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import yfinance as yf
import torch
import torch.nn as nn 


In [51]:
df= yf.download(tickers= 'NFLX', start ='2002-03-11', end= '2023-03-30')

[*********************100%***********************]  1 of 1 completed


In [52]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-05-23,1.156429,1.242857,1.145714,1.196429,1.196429,104790000
2002-05-24,1.214286,1.225000,1.197143,1.210000,1.210000,11104800
2002-05-28,1.213571,1.232143,1.157143,1.157143,1.157143,6609400
2002-05-29,1.164286,1.164286,1.085714,1.103571,1.103571,6757800
2002-05-30,1.107857,1.107857,1.071429,1.071429,1.071429,10154200


In [53]:
closed_prices= df['Close']

In [54]:
seq_len=15

In [55]:
mm= MinMaxScaler()
scaled_price = mm.fit_transform(np.array(closed_prices) [... , None]).squeeze()

In [56]:
scaled_price

array([0.00119131, 0.00121094, 0.00113448, ..., 0.47342547, 0.46743688,
       0.47974673])

In [57]:
X =[]
Y =[]

In [58]:
for i in range(len(scaled_price)- seq_len):
  X.append(scaled_price[i : i + seq_len])
  Y.append(scaled_price[i + seq_len])

In [59]:
X= np.array(X)[..., None]
Y= np.array(Y)[..., None]

In [71]:
train_x= torch.from_numpy(X[:int(0.8 * X.shape[0])]). float()
train_y= torch.from_numpy(Y[:int(0.8 * X.shape[0])]). float()
test_x= torch.from_numpy(X[int(0.8 * X.shape[0]):]). float()
test_y= torch.from_numpy(Y[int(0.8 * X.shape[0]):]). float()

In [78]:
class Model (nn. Module): 
  def __init__(self , input_size, hidden_size): 
    super().__init__()
    self.lstm= nn.LSTM(input_size, hidden_size, batch_first= True)
    self.fc= nn.Linear(hidden_size , 1)
    def forward (self , X): 
      output , (hidden, cell) = self.lstm(x)
      return self.fc(hidden[-1, :])
model = Model(1 , 94)
## jeg skulle have 2 __ på i init 



In [79]:
optimizer = torch.optim.Adam(model.parameters() , lr = 0.001)
loss_fn = nn.MSELoss()



In [80]:
num_epochs = 100 


In [81]:
for epoch in range(num_epochs): 
  output= model(train_x) 
  loss = loss_fn(output, train_y)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch % 10 == 0 and epoch !=0:
    print(epoch , "epoch_loss" , loss.cpu(). numpy())



NotImplementedError: ignored

In [ ]:
model.eval() 
with torch.no_grad():
  output = model(test_x)

pred = mm.inverse_transform(output.numpy())
real = mm.inverse_transform(test_y.numpy())

plt.plot(pred.squeeze(), color="red", label = "predicted")
plt.plot (real.squeeze(), color= "green", label= "real")
plt.show()